In [1]:
# Suppress pandas warnings 
import warnings
warnings.simplefilter(action='ignore', category=Warning)

import pandas as pd

import sqlite3

import yahooquery  
import yfinance as yf
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LogisticRegression

from sklearn.svm import LinearSVC
from sklearn import preprocessing 

from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [79]:
GET_NEW_DATA = False
RUN_KNN_AND_SVM = False

In [5]:
# Relative Strength Index (RSI)
def RSI(df, window=14):
    Gain=df['Close'].copy()
    Loss=df['Close'].copy()
    Avg_gain=df['Close'].copy()
    Avg_loss=df['Close'].copy()
    rsi=df['Close'].copy()

    Gain[:]=0.0
    Loss[:]=0.0
    Avg_gain[:]=0.0
    Avg_loss[:]=0.0
    rsi[:]=0.0
    
    #idx = df['Close']>df['Close'].shift(1)

    for i in range(1,len(df)):
        if df.loc[i,'Close']>df.loc[i-1,'Close']:
            Gain[i]=df.loc[i,'Close']-df.loc[i-1,'Close']
        else:
            # For loss save the absolute value on loss
            Loss[i]=abs(df.loc[i,'Close']-df.loc[i-1,'Close'])
        if i>window:
            Avg_gain[i]=(Avg_gain[i-1]*(window-1)+Gain[i])/window
            Avg_loss[i]=(Avg_loss[i-1]*(window-1)+Loss[i])/window
            rsi[i]=(100*Avg_gain[i]/(Avg_gain[i]+Avg_loss[i])).round(2)
            
    # 50-day simple moving average
    sma_50 = df['Close'].rolling(window=50).mean().round(2)
    
    # 200-day simple moving average
    sma_200 = df['Close'].rolling(window=200).mean()
    
    # 9-day exponential moving average (aka signal line for MACD indicator)
    ema_9 = df['Close'].ewm(span=9, adjust=False).mean().round(2)
    
    # 26-day EMA
    ema_26 = df['Close'].ewm(span=26, adjust=False).mean()
    
    # 12-day EMA
    ema_12 = df['Close'].ewm(span=12, adjust=False).mean()
    
    # MACD = 12-Period EMA − 26-Period EMA
    macd = (ema_12 - ema_26)
    
    # MACD histogram: Difference between MACD line and singal line
    # Positive values as green bars
    # Negative values as red bars
    diff_macd = macd - ema_9
    
    
    return rsi, sma_50, sma_200, ema_9, macd, diff_macd

def get_technical_data(symbols, con):

    # Drop table if exists, to prevent overwriting
    con.execute('DROP table IF EXISTS stock_technical_data')
    
    for symbol in symbols:

        stock = yf.Ticker(symbol)
        df = stock.history(period='2500d', actions=False) # minimum 100 days of data for good RSI accuracy
        df = df.reset_index()
        df.drop(df.columns.difference(['Date','Close']), 1, inplace=True)

        rsi, sma_50, sma_200, ema_9, macd, diff_macd = RSI(df)

        # Add RSI to the dataframe
        df['RSI'] = rsi

        # Add 9-day EMA to the dataframe
        df['EMA_9'] = ema_9

        # Add MACD to the dataframe
        df['MACD'] = macd.round(2)

        # Add signal to the dataframe
        df['signal'] = macd.ewm(span=9, adjust=False).mean().round(2)

        # Add diff_MACD to the dataframe
        df['diff_MACD'] = (df['MACD'] - df['signal'])

        # Add ticker symbol to the dataframe
        df['ticker'] = symbol

        # Write dataframe to database
        df.to_sql("stock_technical_data", con, if_exists="append")

def get_fundamental_data(symbols, con):

    # Drop table if exists, to prevent overwriting
    con.execute('DROP table IF EXISTS stock_fundamental_data')    
    
    for symbol in symbols:

        stock = yf.Ticker(symbol)

        # Get revenue and earnings data
        df_earnings = stock.earnings

        # Get R&D and Income Before Tax data
        df_financials = stock.financials
        try:
            df_financials.columns = df_financials.columns.year # Change to year only format
        except:
            print('symbol '+symbol+' issue')
            continue

        df_financials = df_financials.T # transpose for years as rows
        # Drop all columns except R&D and IBT
        df_financials.drop(df_financials.columns.difference(['Research Development', 'Income Before Tax']), 1, inplace=True)
        df_financials = df_financials.iloc[::-1] # Reverse the order

        # Merge the dataframes
        df = df_earnings.merge(df_financials, how='outer', left_index=True, right_index=True)

        # Add ticker symbol to the dataframe
        df['ticker'] = symbol

        # Write dataframe to database
        try:
            df.to_sql("stock_fundamental_data", con, if_exists="append")
        except:
            print('Unable to write '+symbol+' to database')
            continue
        
def get_financial_data(symbols):
    sp500 = yahooquery.Ticker(symbols, asynchronous=False)
    sp500_all_financial_data = sp500.all_financial_data()
    sp500_all_financial_data['ResearchAndDevelopmentPerShare'] = sp500_all_financial_data['ResearchAndDevelopment']/sp500_all_financial_data['BasicAverageShares']
    sp500_all_financial_data['TotalRevenuePerShare'] = sp500_all_financial_data['TotalRevenue']/sp500_all_financial_data['BasicAverageShares']
    sp500_all_financial_data['DebtToEquity'] = sp500_all_financial_data['TotalDebt']/sp500_all_financial_data['StockholdersEquity']

    sp500_financial_data = sp500_all_financial_data[['asOfDate',
                                                     'BasicAverageShares',
                                                     'BasicEPS', 
                                                     'ResearchAndDevelopmentPerShare',
                                                     'TotalRevenuePerShare',
                                                     'DebtToEquity']]

    sp500_financial_data = sp500_financial_data.reset_index()
    
    return sp500_financial_data

In [62]:
#get symbols
symbols = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]['Symbol'].values.tolist()

con = sqlite3.connect('stock_data_500.db')

if GET_NEW_DATA:
    get_technical_data(symbols, con)
    get_fundamental_data(symbols, con)

In [7]:
df_tech = pd.read_sql("""SELECT * FROM stock_technical_data""",con)
df_tech['Date'] = pd.to_datetime(df_tech['Date'])

In [8]:
if GET_NEW_DATA:
    sp500_financial_data = get_finanical_data(symbols)
    df_all = df_tech.merge(
        sp500_financial_data,
        how = 'left',
        left_on=['ticker','Date'],
        right_on=['symbol','asOfDate']
    )
    df_all['Price/Earnings Ratio'] = df_all['Close']/df_all['BasicEPS']
    df_all['Price/Revenue Ratio'] = df_all['Close']/df_all['TotalRevenuePerShare']
    df_all['Research/Revenue Ratio'] = df_all['ResearchAndDevelopmentPerShare']/df_all['TotalRevenuePerShare']
    
    df_all = df_all.drop(
        columns=[
            'symbol',
            'asOfDate',
            'BasicAverageShares',
            'BasicEPS', 
            'ResearchAndDevelopmentPerShare',
            'TotalRevenuePerShare',
        ]
    )
    
    df_all.to_sql("stock_all_financial_data", con, if_exists="replace")

In [9]:
df_all = pd.read_sql("""SELECT * FROM stock_all_financial_data""",con)
df = df_all.copy()
df2 = df_all.copy()

In [56]:
# Create the input dataframe with RSI and MACD
df['return10']=(df['Close'].shift(-10) - df['Close'])*100/df['Close']

# Remove rows with RSI <10
df = df[df['RSI']>=10]

# Drop all rows with Nan
df = df.dropna(subset=['return10'])

x=df[['RSI','diff_MACD']].values

df['y10'] = df['return10'].apply(lambda x:1 if x>0 else 0)

In [57]:
# x_var_list = ['RSI','diff_MACD','DebtToEquity',
#        'Price/Earnings Ratio','Price/Revenue Ratio']
x_var_list = ['RSI','DebtToEquity','Price/Revenue Ratio']


# Create the input dataframe with RSI, MACD, Debt-to-Equity, Price-to-Earnings and Price-to-Revenue
df2['return10']=(df2['Close'].shift(-10) - df2['Close'])*100/df2['Close']

# Remove rows with RSI <10
df2 = df2[df2['RSI']>=10]

# Drop all rows with Nan
df2 = df2.dropna(subset=['return10','DebtToEquity',
       'Price/Earnings Ratio','Price/Revenue Ratio'])

x2=df2[x_var_list].values

df2['y10'] = df2['return10'].apply(lambda x:1 if x>0 else 0)

In [58]:
# Using 10-day returns to create final logistic regression model using only RSI and MACD as predictors
y=df['y10'].values.reshape(-1,1)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
logmodel=LogisticRegression()
logmodel.fit(X_train,y_train)
logmodel


predictions=logmodel.predict(X_test)
print('Logistic Regression Accuracy score of 10 day returns is: '+str(accuracy_score(y_test,predictions)))

if RUN_KNN_AND_SVM:
    knn = KNeighborsClassifier()
    knn.fit(X_train,y_train)

    predictions=knn.predict(X_test)
    print('KNN Accuracy score of 10 day returns is: '+str(accuracy_score(y_test,predictions)))


    linsvc = LinearSVC()
    scaler = preprocessing.StandardScaler().fit(X_train)

    X_train = scaler.transform(X_train)
    linsvc.fit(X_train,y_train)

    predictions=linsvc.predict(scaler.transform(X_test))
    print('Linear Support Vector Classification Accuracy score of 10 day returns is: '+str(accuracy_score(y_test,predictions)))


Logistic Regression Accuracy score of 10 day returns is: 0.5843259655604391


In [80]:
# Using 10-day returns to create final logistic regression model using RSI, MACD, Debt-to-Equity,
# Price-to-Earnings and Price-to-Revenue

y=df2['y10'].values.reshape(-1,1)
X_train, X_test, y_train, y_test = train_test_split(x2, y, test_size=0.3, random_state=42)
logmodel=LogisticRegression()
logmodel.fit(X_train,y_train)
logmodel

predictions=logmodel.predict(X_test)
print('Logistic Regression Accuracy score of 10 day returns is: '+str(accuracy_score(y_test,predictions)))

if RUN_KNN_AND_SVM:

    knn = KNeighborsClassifier()
    knn.fit(X_train,y_train)

    predictions=knn.predict(X_test)
    print('KNN Accuracy score of 10 day returns is: '+str(accuracy_score(y_test,predictions)))


    linsvc = LinearSVC()
    scaler = preprocessing.StandardScaler().fit(X_train)

    X_train = scaler.transform(X_train)
    linsvc.fit(X_train,y_train)

    predictions=linsvc.predict(scaler.transform(X_test))
    print('Linear Support Vector Classification Accuracy score of 10 day returns is: '+str(accuracy_score(y_test,predictions)))

Logistic Regression Accuracy score of 10 day returns is: 0.583259294875145
KNN Accuracy score of 10 day returns is: 0.5456616947664391
Linear Support Vector Classification Accuracy score of 10 day returns is: 0.583259294875145


In [60]:

last_day = df_all['Date'].max()
df_last_day = df_all[df_all['Date'] == last_day]
df_last_day = df_last_day.dropna(subset=x_var_list)

x_last_day = df_last_day[x_var_list]

predictions=logmodel.predict(x_last_day)
#print(predictions)
df_last_day['prob'] = logmodel.predict_proba(x_last_day)[:,1]

In [63]:
df_last_day = df_last_day.sort_values('prob', ascending=False)
df_top_ten = df_last_day.iloc[0:10][['ticker',  'prob']]
df_top_ten.to_sql("top_ten_list", con, if_exists="replace")

In [64]:
df_top_ten

,ticker,prob
721441,MKTX,0.630031
873574,PAYC,0.623780
875189,PYPL,0.616979
590958,ILMN,0.615330
344434,XRAY,0.615020
476518,FLT,0.614735
1164538,DIS,0.614352
14744,ATVI,0.613761
733941,MA,0.612933
524256,GPN,0.612579


#### Backtest last 30 days

In [65]:
df = pd.read_sql("""SELECT * FROM stock_all_financial_data""",con)

In [66]:
# grab the date of 10th last day
last_10_date = df['Date'].iloc[-11,]

# 10-day returns
df['return10']=(df['Close'].shift(-10) - df['Close'])*100/df['Close']

# y=1 if return is positive, else 0
df['y10'] = df['return10'].apply(lambda x:1 if x>0 else 0)

# Drop all rows with Nan
df = df.dropna(subset=['return10','DebtToEquity',
       'Price/Earnings Ratio','Price/Revenue Ratio'])

# Remove rows with RSI <10
df = df[df['RSI']>=10]

df = df[df['Date']<=last_10_date]

In [67]:
last_30_date = df['Date'].iloc[-30,]
df_evaluate = df[df['Date']>=last_30_date] # Evaluation dataset of last 30 days
df_test = df[df['Date']<last_30_date] # Test dataset for regression model

In [68]:
# Train logistic regression model on test dataset
x=df_test[x_var_list].values

y=df_test['y10'].values.reshape(-1,1)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
logmodel=LogisticRegression()
logmodel.fit(X_train,y_train)
logmodel

predictions=logmodel.predict(X_test)
#print('Accuracy score of 10 day returns is: '+str(accuracy_score(y_test,predictions)))

In [70]:
avg_return=[]
correct_predictions=[]

# loop through 30 days
for i in range(30):
    #print(df_evaluate['Date'].iloc[i,])
    today = df_evaluate['Date'].iloc[i,] # each days date
    print('Date:', today)
    df_today = df_evaluate[df_evaluate['Date']==today]
    #print(len(df_today)) #verify there are 451 stocks
    x_today = df_today[x_var_list].values
    probs=logmodel.predict_proba(x_today) # predict probabilities

    probs = probs.T #transpose
    largest_probs = probs[1].argsort()[-10:][::-1]
    top_tickers=[]
    for n in largest_probs:
        top_tickers.append(df_today.iloc[n,9])
    #print('Day '+ str(i+1) + ' top tickers: ' + str(top_tickers))
    #print(top_tickers)

    df_top = df_today[df_today['ticker'].isin(top_tickers)]
    avg_return.append(df_top['return10'].mean())
    print('Day '+ str(i+1) + ' average return: ' + str(round(df_top['return10'].mean(),2)))
    
    correct_predictions.append(df_top['y10'].sum()/10)
    print('Day '+ str(i+1) + ' % correct predictions: ' + str(df_top['y10'].sum()*10))
    print('\n')

Date: 2021-10-04 00:00:00
Day 1 average return: 5.48
Day 1 % correct predictions: 80


Date: 2021-10-05 00:00:00
Day 2 average return: 4.5
Day 2 % correct predictions: 80


Date: 2021-10-06 00:00:00
Day 3 average return: 4.08
Day 3 % correct predictions: 80


Date: 2021-10-07 00:00:00
Day 4 average return: 4.63
Day 4 % correct predictions: 90


Date: 2021-10-08 00:00:00
Day 5 average return: 6.15
Day 5 % correct predictions: 90


Date: 2021-10-11 00:00:00
Day 6 average return: 6.07
Day 6 % correct predictions: 90


Date: 2021-10-12 00:00:00
Day 7 average return: 5.61
Day 7 % correct predictions: 90


Date: 2021-10-13 00:00:00
Day 8 average return: 3.6
Day 8 % correct predictions: 90


Date: 2021-10-14 00:00:00
Day 9 average return: 3.43
Day 9 % correct predictions: 80


Date: 2021-10-15 00:00:00
Day 10 average return: 5.86
Day 10 % correct predictions: 100


Date: 2021-10-18 00:00:00
Day 11 average return: 6.2
Day 11 % correct predictions: 100


Date: 2021-10-19 00:00:00
Day 12 average

In [71]:
# Grab SPY data for benchmarking
tickers = 'SPY'
stock = yf.Ticker(tickers)
df_spy = stock.history(start="2021-10-04", end="2021-11-30").reset_index()
# 10-day returns
df_spy['return10']=(df_spy['Close'].shift(-10) - df_spy['Close'])*100/df_spy['Close']
df_spy = df_spy[df_spy['Date']<='2021-11-12']

In [72]:
x_axis=list(range(1,31))
plt.figure(figsize=(20, 10), dpi=1000)
plt.plot(df_spy['Date'], avg_return, label='Logit model avg return')
plt.plot(df_spy['Date'], df_spy['return10'], label='SPY')
plt.title('Average returns over 10 day hold', fontsize=24)
plt.xlabel('Buy Date', fontsize=18)
plt.ylabel('% Return (when sold after 10 days)', fontsize=18)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.legend(loc="upper right", fontsize=16)
plt.show()

In [73]:
#close connection
con.close()

In [75]:
# import statsmodels.api as sm
# logit_model=sm.Logit(y_train,X_train)
# result=logit_model.fit()
# print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.680080
         Iterations 4
                          Results: Logit
Model:              Logit            Pseudo R-squared: -0.002     
Dependent Variable: y                AIC:              286076.0853
Date:               2021-12-05 16:36 BIC:              286106.8544
No. Observations:   210321           Log-Likelihood:   -1.4304e+05
Df Model:           2                LL-Null:          -1.4279e+05
Df Residuals:       210318           LLR p-value:      1.0000     
Converged:          1.0000           Scale:            1.0000     
No. Iterations:     4.0000                                        
---------------------------------------------------------------------
          Coef.     Std.Err.       z       P>|z|     [0.025    0.975]
---------------------------------------------------------------------
x1        0.0052      0.0001    46.4376    0.0000    0.0050    0.0054
x2        0.0003      0.0002     2.1523  